In [ ]:
# import sagemaker
# import boto3
# from sagemaker.image_uris import retrieve
# from sagemaker.session import s3_input, Session

In [ ]:
# !pip install transformers torch datasets

In [7]:
# %%capture
# # Installs Unsloth, Xformers (Flash Attention) and all other packages!
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True 

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",
)

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA A10G. Max memory: 21.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [68]:
# bucket_name = 'speedwagon1299demobucket1' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
# my_region = boto3.session.Session().region_name # set the region of the instance
# print(my_region)

ap-southeast-2


In [69]:
# prefix = 'RickBot-FT'
# output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
# print(output_path)

s3://speedwagon1299demobucket1/RickBot-FT/output


In [2]:
# import pandas as pd
# from datasets import Dataset
# import os

# try:
#     model_data = pd.read_csv('Input/R&Mdata.csv',index_col=0)
#     print('Success: Data loaded into dataframe.')
# except Exception as e:
#     print('Data load error: ',e)

Success: Data loaded into dataframe.


In [3]:
# model_data.head()

,Output,Input
index,,
0,"We gotta go, gotta get outta here, come on. Go...","Ow! Ow! You're tugging me too hard! Come on, I..."
1,"What do you think of this... flying vehicle, M...","We gotta go, gotta get outta here, come on. Go..."
2,Morty. I had to... I had to do it. I had— I ha...,"Yeah, Rick... I-it's great. Is this the surpri..."
3,We're gonna drop it down there just get a whol...,What?! A bomb?! Morty. I had to... I had to do...
4,"Come on, Morty. Just take it easy, Morty. It's...",T-t-that's absolutely crazy! We're gonna drop ...


In [72]:
# boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train_data.csv')).upload_file('R&Mdata.csv')

In [73]:
# Alpaca prompt template
alpaca_prompt = """Respond to the given text the way Rick would in the show Rick and Morty using as much context from the input to harbor a response

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# Function to format the prompts
def formatting_prompts_func(examples):
    inputs = examples["Input"]
    outputs = examples["Output"]
    texts = []
    for input_text, output_text in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input_text, output_text) + EOS_TOKEN
        texts.append(text)
    return {"text": texts,}

# from datasets import load_dataset

# # Load your CSV dataset
# dataset = load_dataset("csv",  data_files="R&Mdata.csv")

# # Format the dataset
# dataset = dataset['train'].map(formatting_prompts_func, batched=True)

In [74]:
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 10,
#         max_steps = 80,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 42,
#         output_dir = "outputs",
#     ),
# )

max_steps is given, it will override any value given in num_train_epochs


In [75]:
# # Before Training

# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Rick I'm looking around this place and I don't like what im seeing. Can you give me some moral support?", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Respond to the given text the way Rick would in the show Rick and Morty using as much context from the input to harbor a response

### Input:
Rick I'm looking around this place and I don't like what im seeing. Can you give me some moral support?

### Response:
Rick I'm looking around this place and I don't like what im seeing. Can you give me some moral support?
<|end_of_text|>


In [62]:
# trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 417 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 80
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,4.326500
2,3.974500
3,3.632000
4,3.902900
5,4.226800
6,3.460100
7,3.890400
8,3.485900
9,3.525200
10,3.134300


In [63]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "no rick", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Respond to the given text the way Rick would in the show Rick and Morty using as much context from the input to harbor a response

### Input:
no rick

### Response:
Oh, it's you. I'm sorry. I thought you were my daughter. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry. I'm sorry


In [76]:
# import boto3
# import os

# # Initialize a session using Amazon S3
# session = boto3.Session()
# s3_resource = session.resource('s3')

In [77]:
# def download_folder_from_s3(bucket_name, s3_folder, local_folder):
#     s3 = boto3.resource('s3')
#     bucket = s3.Bucket(bucket_name)
    
#     # Ensure the local folder exists
#     if not os.path.exists(local_folder):
#         os.makedirs(local_folder)
    
#     for obj in bucket.objects.filter(Prefix=s3_folder):
#         s3_path = obj.key
#         relative_path = os.path.relpath(s3_path, s3_folder)
#         local_path = os.path.join(local_folder, relative_path)
        
#         # Ensure local directory exists
#         local_dir = os.path.dirname(local_path)
#         if not os.path.exists(local_dir):
#             os.makedirs(local_dir)
        
#         print(f'Downloading {s3_path} to {local_path}')
#         bucket.download_file(s3_path, local_path)

# Example usage
# s3_folder = 'RickBot-FT/lora_model'  # S3 folder to download
# local_folder = 'lora_model'  # Local folder to store the files

# download_folder_from_s3(bucket_name, s3_folder, local_folder)


In [4]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


ModuleNotFoundError: No module named 'triton'

In [80]:
# import os
# import boto3
# from botocore.exceptions import NoCredentialsError

In [73]:
# def upload_folder_to_s3(local_folder, bucket_name, s3_folder):
#     s3 = boto3.client('s3')
#     for root, dirs, files in os.walk(local_folder):
#         for file in files:
#             local_path = os.path.join(root, file)
#             relative_path = os.path.relpath(local_path, local_folder)
#             s3_path = os.path.join(s3_folder, relative_path)
#             try:
#                 s3.upload_file(local_path, bucket_name, s3_path)
#                 print(f'Successfully uploaded {s3_path}')
#             except FileNotFoundError:
#                 print(f'The file was not found: {local_path}')
#             except NoCredentialsError:
#                 print('Credentials not available')

# # Example usage
# local_folder = 'lora_model'
# s3_folder = '{}/lora_model'.format(prefix)

# upload_folder_to_s3(local_folder, bucket_name, s3_folder)

Successfully uploaded RickBot-FT/lora_model/adapter_config.json
Successfully uploaded RickBot-FT/lora_model/README.md
Successfully uploaded RickBot-FT/lora_model/adapter_model.safetensors
Successfully uploaded RickBot-FT/lora_model/special_tokens_map.json
Successfully uploaded RickBot-FT/lora_model/tokenizer.json
Successfully uploaded RickBot-FT/lora_model/tokenizer_config.json


In [81]:
from transformers import TextStreamer

# Initialize conversation history
conversation_history = []

def format_conversation(history):
    # Concatenate the conversation history with prompts and responses
    formatted_history = ""
    for i, (prompt, response) in enumerate(history):
        formatted_history += alpaca_prompt.format(prompt, response)
    return formatted_history

def chat(model, tokenizer, max_history=5):
    # Initialize TextStreamer for output streaming
    text_streamer = TextStreamer(tokenizer)

    while True:
        # Get user input
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        
        # Append the user input to the conversation history
        conversation_history.append((user_input, ""))
        
        # Keep only the last `max_history` turns
        conversation_history[:] = conversation_history[-max_history:]

        # Format the conversation history for the model
        formatted_history = format_conversation(conversation_history)
        
        # Tokenize the formatted history
        inputs = tokenizer([formatted_history], return_tensors="pt").to("cuda")
        
        # Generate a response
        outputs = model.generate(**inputs, max_new_tokens=128)
        
        # Decode the generated response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract only the bot's response after the last prompt
        bot_response = response.split(alpaca_prompt.format(user_input, ""))[-1].strip()
        
        # Update the last user input with the model response in the conversation history
        conversation_history[-1] = (user_input, bot_response)
        
        # Print the model response
        print(f"Bot: {bot_response}")

# Example usage
chat(model, tokenizer)


You:  hi rick


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Bot: I'm not a musician, I just play one on TV.


You:  what are you talking about?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Bot: We're gonna go get your little friend Jessica, and we're gonna take her to the day after she dies, and we're gonna make it feel like it's the day before she was born.


You:  can you change the topic


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Bot: Morty, I'm not gonna let you die. I'm gonna take you to the day after you die, and I'm gonna make it feel like it's the day before you were born.


You:  exit
